## Screening task 4

The problem statement is to find the lowest eigenvalue of matrix H:
$\begin{bmatrix}1 & 0 & 0 & 0 \\0 & 0 & -1 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}$
with a custom VQE implementation

The first step consists on decomposing the H matrix into a sum of Pauli matrices, so it can be mapped with quantum gates X,Y,Z.

In [49]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.operation import Observable
import torch

In [50]:
H = np.array([[1,0,0,0],[0,0,-1,0],[0,-1,0,0],[0,0,0,1]])
coeffs ,obs = DecomposeIntoPauliTerms(H)
Hamil = qml.Hamiltonian(coeffs, obs)
print(Hamil)

(0.5) [I0 I1]
+ (-0.5) [X0 X1]
+ (-0.5) [Y0 Y1]
+ (0.5) [Z0 Z1]


<ipython-input-46-d58ea43f47d6>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  a_ij = np.float(0.25 * HS(kron(S[i], S[j]), H))


In [46]:
## Pennylane code

def circuit(params,**kwargs):

    qml.Hadamard(wires=0) 
    qml.CNOT(wires=[0, 1])
    qml.RX(params[0], wires=0)


def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def c2s(c):
    """Return a string representation of a complex number c"""
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def DecomposeIntoPauliTerms(H):
    
    """Decompose Hermitian matrix H acting on 2 qubits into list of 2-qubit Pauli observable
    Returns  float coeffs and observable class (Pauli operations) list
    """
    n = int(np.log2(len(H)))
    
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    
    paulis = [qml.Identity, qml.PauliX, qml.PauliY, qml.PauliZ] # These are Pennylane operations  
    coefs = []
    obs = []
    for i in range(2**n):
        for j in range(2**n):
            a_ij = np.float(0.25 * HS(kron(S[i], S[j]), H))
            if a_ij != 0.0:
                term0 = paulis[i]
                term1 = paulis[j]
                pauli_term = term0(0) @ term1(1)
                obs.append(pauli_term)
                coefs.append(a_ij)
                
    return coefs,obs

In [53]:

nr_qubits = 2
dev = qml.device("default.qubit", wires=nr_qubits)

cost_fn = qml.VQECost(circuit, Hamil, dev)
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.rand(1)


max_iterations = 200
conv_tol = 1e-012

prev_energy = cost_fn(params)

for n in range(max_iterations):
    
    params = opt.step(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print('Iteration = {:},  Ground-state energy = {:.8f},  Convergence parameter = {'
              ':.8f} '.format(n, energy, conv))

    if conv <= conv_tol:
        break

    prev_energy = energy

print()
print('Final convergence parameter = {:.8f} '.format(conv))
print('Final value of the ground-state energy = {:.8f}'.format(energy))
print()
print('Final circuit parameters = \n', params)

Iteration = 0,  Ground-state energy = 0.72656729,  Convergence parameter = 0.12657680 
Iteration = 20,  Ground-state energy = -0.99999995,  Convergence parameter = 0.00000010 

Final convergence parameter = 0.00000000 
Final value of the ground-state energy = -1.00000000

Final circuit parameters = 
 [3.14159194]


### Sources